In [9]:
# import necessary libraries
import pandas as pd
import numpy as np
from gensim import corpora
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from collections import Counter
from itertools import product
from gensim.models import CoherenceModel
from wordcloud import WordCloud
from PIL import Image
import matplotlib.pyplot as plt
from gensim.utils import simple_preprocess
from pprint import pprint

from bitermplus import BTM

import bitermplus as btm
import tmplot as tmp
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from gensim.models import CoherenceModel
from sklearn.model_selection import ParameterGrid
from gensim.models.coherencemodel import CoherenceModel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline


import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import re
import hdbscan
import umap


from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from transformers import DistilBertTokenizer, DistilBertModel

import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel


In [ ]:

#LDA==========================================================================================================
#https://radimrehurek.com/gensim/models/ldamodel.html

#read the txt file
with open(r'cleaned_tweets.txt') as file:
    data = file.readlines()

# create a dataframe
df = pd.DataFrame(data, columns=['text'])

# write dataframe to csv file
df.to_csv('cleaned_tweets.csv', index=False)
data = pd.read_csv('cleaned_tweets.csv')
# preprocess text data
stop_words = stopwords.words('English')
TL_stopwords = open("Stopwords_EN_TL.txt").read().splitlines() 
stop_words.extend(TL_stopwords)
def preprocess(text):
    result = []
    for token in simple_preprocess(text):
        if token not in stop_words:
            result.append(token)
    return result

data['processed_text'] = data['text'].apply(preprocess)

# # Join the different processed titles together.
# long_string = ','.join(data['text'] )
# # Create a WordCloud object
# wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# # Generate a word cloud
# wordcloud.generate(long_string)
# # Visualize the word cloud
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

# Tokenize the documents
tokenized_docs = [doc.split() for doc in data]
# create dictionary and corpus
dictionary = corpora.Dictionary(data['processed_text'])
corpus = [dictionary.doc2bow(text) for text in data['processed_text']]
#  # Define the range of hyperparameter values to search over
# num_topics_list = [3,5,10]
# passes_list = [3,5,10]
# alpha_list = ['symmetric',0.3,0.5]

# # Perform a grid search over the hyperparameter values
# results = []
# for num_topics, passes , alpha in product(num_topics_list, passes_list,alpha_list):
#     lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=passes,alpha=alpha,)
#     coherence_model = CoherenceModel(model=lda_model, texts=data['processed_text'], dictionary=dictionary, coherence='c_v')
#     coherence_score = coherence_model.get_coherence()
#     results.append({'num_topics': num_topics,'alpha':alpha, 'passes': passes, 'coherence_score': coherence_score})

# #Find the hyperparameters with the highest coherence score
# best_params = max(results, key=lambda x: x['coherence_score'])
# print('Best parameters:', best_params)


#Best parameters: {'num_topics': 3, 'alpha': 0.3, 'passes': 10, 'coherence_score': 0.6115592649171243} runtime:4omins

In [ ]:
#train LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, alpha=0.3,passes=10)

# Compute the coherence score for the trained LDA model
coherence_model = CoherenceModel(model=lda_model, texts=data['processed_text'], dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

print('Coherence score:', coherence_score)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
#Biterm========================================================================================================
#https://bitermplus.readthedocs.io/en/latest/index.html
#IMPORTING DATA
df = pd.read_csv(
    'cleaned_tweets.csv', header=None, names=['texts'])
# preprocess text data
stop_words = stopwords.words('English')
TL_stopwords = open("Stopwords_EN_TL.txt").read().splitlines() 
stop_words.extend(TL_stopwords)
texts = df['texts'].str.strip().tolist()
# Obtaining terms frequency in a sparse matrix and corpus vocabulary
X, vocabulary, vocab_dict = btm.get_words_freqs(texts,stop_words=stop_words)
tf = np.array(X.sum(axis=0)).ravel()
# Vectorizing documents
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
docs_lens = list(map(len, docs_vec))
# Generating biterms
biterms = btm.get_biterms(docs_vec)
#after using grid search the best param was:
#M=10, alpha=0.01, beta=0.01 = coherence(-6435),perplexity(5020)



In [ ]:
# INITIALIZING AND RUNNING MODEL
model = btm.BTM(
    X, vocabulary, T=10, M=10, alpha=0.01, beta=0.01)
model.fit(biterms, iterations=20)
p_zd = model.transform(docs_vec)

#METRICS
perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 8)
coherence = btm.coherence(model.matrix_topics_words_, X, M=20)

topics = btm.get_top_topic_words(
    model,
    words_num=10,)
print(topics)


In [8]:
#distilbert===================================================================================================

# load data into a pandas dataframe
df = pd.read_csv('cleaned_tweets.csv')
# clean data
df['text'] = df['text'].apply(lambda x: x.lower())

# tokenize text data using DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
df['text'] = df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
# extract embeddings using DistilBERT model
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
embeddings = []
for text in df['text']:
    input_ids = torch.tensor([text])
    with torch.no_grad():
        output = model(input_ids)
        embeddings.append(output[0][0][0].numpy())
df['embeddings'] = embeddings
# reduce dimensionality of embeddings using UMAP
reducer = umap.UMAP(n_neighbors=15, n_components=5, metric='cosine')
umap_embeddings = reducer.fit_transform(df['embeddings'].tolist())
df['umap_embeddings'] = umap_embeddings.tolist()
# preprocess data
df['text'] = df['text'].apply(lambda x: str(x).lower())

# perform clustering using HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=15, metric='euclidean')
clusters = clusterer.fit_predict(df['umap_embeddings'].tolist())
df['cluster'] = clusters

# print top 10 words for each cluster
stop_words = stopwords.words('English')
TL_stopwords = open("Stopwords_EN_TL.txt").read().splitlines() 
stop_words.extend(TL_stopwords)
if 'cluster' in df.columns:
    for cluster in sorted(df['cluster'].unique()):
        texts = df[df['cluster'] == cluster]['text']
        if len(texts) > 1:
            vectorizer = CountVectorizer(stop_words=TL_stopwords, max_features=5000)
            X = vectorizer.fit_transform(texts)
            lda = LatentDirichletAllocation(n_components=1, random_state=42)
            lda.fit(X)
            feature_names = vectorizer.get_feature_names()
            top_words = [feature_names[i] for i in lda.components_[0].argsort()[:-11:-1]]
            print(f'Topic {cluster}: {" | ".join(top_words)}')
else:
    print('Error: "cluster" column not found in DataFrame.')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: name 'stop_words' is not defined

In [ ]:
print("\n\n==========================================COHERENCE=============================================\n\n")

#Reg exp for tokenizing the data set
tokenizer = lambda s: re.findall('\w+', s.lower())
text = [tokenizer(t) for t in data]

# Getting Topics
all_topics = topic_model.get_topics()
top = []
keys = []
for x in range(10):
    keys.append(freq['Topic'].head(10)[x])

#Tokenizing
prefix = 'Getting Topics'
pbar2 = tqdm(total=len(keys), position=0, leave=True)
pbar2.set_description(prefix)
for key in tqdm(keys, desc='Getting Topics', position=0, leave=True):
    values = all_topics[key]
    topic_1 = []
    for value in tqdm(values, desc='Retrieving Values in topic ' + str(key), position=0, leave=True):
        topic_1.append(value[0])
    top.append(topic_1)

# Creating a dictionary with the vocabulary
word2id = Dictionary(text)
vec = CountVectorizer()
X = vec.fit_transform(data).toarray()
vocab = np.array(vec.get_feature_names())
# Coherence model
cm = CoherenceModel(topics=top, texts=text, coherence='u_mass', dictionary=word2id)
coherence_per_topic = cm.get_coherence_per_topic()
#Results
print("\n==========================================COHERENCE RESULTS=============================================\n")
for index, x in enumerate(coherence_per_topic):
    print("topic %2d : %5.2f" % (index + 1, x))

coherence = cm.get_coherence()
print(coherence)



In [ ]:
#bertopic===================================================================================================

# Read Stopwords_EN_TL.txt and save it into a pandas DataFrame
stop_words_dataframe = pd.read_csv("Stopwords_EN_TL.txt")
stop_words = set(stop_words_dataframe.iloc[:,0])
# Read csv and save into a pandas DataFrame
docs_dataframe = pd.read_csv("cleaned_tweets.txt")
# Remove stopwords for every comment and clean the dataset
docs = []
index = 0
for w in docs_dataframe.iloc[:,0].items():
    series = hero.remove_stopwords(pd.Series(w[1]),stop_words)
    series = hero.preprocessing.clean(series)
    docs.append(series[0])
# Output the cleaned dataset to an excel file
cleaned_dataset = pd.DataFrame(docs)
cleaned_dataset.to_excel("cleaned_tweets.xlsx")
# Initialize the model and fit it to the data

# Hyperparameters:
# language - "english" or "multilingual"
# top_n_words - the top_n_words in each topic (no effect)
# n_gram_range - the n-gram to be used by the vectorizer in the model (no effect / incoherent)
# min_topic_size - how big a topic should be, adjusted to be similar to LDA
# nr_topics - topic reduction, made topics more incoherent

topic_model = BERTopic(min_topic_size=25, language = "multilingual")
topics, probs = topic_model.fit_transform(docs)
# Print the topics found by the model
topics = topic_model.get_topic_info()
topics.to_excel("output.xlsx")
topics
# Extract vectorizer and tokenizer from BERTopic
vectorizer = topic_model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation
tokens = [tokenizer(doc) for doc in docs]
dictionary = corpora.Dictionary(tokens)

topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words,
                                 texts=tokens,
                                 dictionary=dictionary, 
                                 coherence='c_v')

# Print Coherence
coherence = coherence_model.get_coherence()
coherence
topic_model.visualize_barchart()